In [1]:
import synapticflow as sf
import numpy as np 
import random
import torch

### generate connections

In [2]:
input_layer = sf.LIFPopulation(n = 4, refrac_length=0, dt = 1)
output_layer = sf.LIFPopulation(n = 10, refrac_length=0, dt = 1)

In [3]:
network = sf.Connection(pre = input_layer, post = output_layer, w_min = 1, w_max = 10)

In [4]:
def BCDConversion(n) :

	if (n == 0) :
		return [0, 0, 0, 0]

	rev = 0
	while (n > 0) :

		rev = rev * 10 + (n % 10)
		n = n // 10
		
	while (rev > 0) :

		b = str(rev % 10)
		
		out = "{0:04b}".format(int(b, 16))
        
		rev = rev // 10
	
	number_list = list(out)
	for i in range(len(number_list)):
		number_list[i] = int(number_list[i])

	return number_list

In [5]:
# function for generating the encoded current
def input_encoding():
    numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    random_sample = random.choice(numbers)
    
    encoded_current = BCDConversion(random_sample)
    encoded_current = [i * 10 for i in encoded_current]
   
    return torch.tensor(encoded_current), random_sample

In [6]:
def output_decoding(output):
    res = []
    for i in range(len(output)):
        if output[i] == 1:
            res.append(output[i] * i)
    return res

## learning process

In [7]:
def process(network, input_layer, output_layer, epochs):
    learning = sf.MSTDP(lr = [0.0003, 0.0002], connection=network)

    for i in range(epochs):
        encoded_current, sample = input_encoding()
        input_layer.forward(encoded_current)
        input_spikes = input_layer.s
        second_input = network.compute(input_spikes)
        output_layer.forward(second_input)
        out_spikes = output_layer.s
        decoded_output = output_decoding(out_spikes)
        input_layer.reset_state_variables()
        output_layer.reset_state_variables()
        network.reset_state_variables()
        if len(decoded_output) == 0:
            reward = -1000
        elif len(decoded_output) == 1 and sample in decoded_output:
            reward = 1000
        else:
            reward = -80

        if (sample in decoded_output):
            reward = 100 - ((len(decoded_output) - 1) * 10)
        else:
            reward = -90
        learning.update(reward = reward)

In [8]:
process(network, input_layer, output_layer, 1000000)

## testing model accuracy

In [9]:
def model_accuracy(network, input_layer, output_layer):
    predictions = {'exact': 0, 'semi_accurate': 0}
    for i in range(10000):
        input_current, sample = input_encoding()
        input_layer.forward(input_current)
        in_spikes = input_layer.s
        second_input = network.compute(in_spikes)
        output_layer.forward(second_input)
        out_spikes = output_layer.s
        decoded_output = output_decoding(out_spikes)
        input_layer.reset_state_variables()
        output_layer.reset_state_variables()
        network.reset_state_variables()

        if len(decoded_output) == 1 and sample in decoded_output:
            predictions.update({'exact': predictions.get('exact') + 1})

        if (sample in decoded_output):
            predictions.update({'semi_accurate': 
            predictions.get('semi_accurate') + 1})
    e_accuracy = predictions.get('exact') / 10000
    s_accuracy = predictions.get('semi_accurate') / 10000
    totall_accuracy = e_accuracy + s_accuracy
    return e_accuracy, s_accuracy, totall_accuracy

In [11]:
e_accuracy, s_accuracy, total_accuracy = model_accuracy(network, input_layer, output_layer)
print(f"model exact estimation accuracy : {e_accuracy}\n" + 
      f"model semi_accurate percentage: {s_accuracy}\n" + 
      f"model total accuracy: {total_accuracy}")

model exact estimation accuracy : 0.0
model semi_accurate percentage: 0.4887
model total accuracy: 0.4887
